In [2]:
import pandas as pd
import folium
import geopandas as gpd
from shapely.geometry import Polygon, Point
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import time

In [3]:
df1 = pd.read_csv("data/df_large.csv",index_col=False)
df2 = pd.read_csv("data/df_small.csv",index_col=False)

In [4]:
df1_unigue = df1[['LAT', 'LON']].drop_duplicates()
df2_unigue = df2[['LAT', 'LON']].drop_duplicates()

In [5]:
display(len(df1_unigue))
display(df1_unigue.head())
display(len(df2_unigue))
display(df2_unigue.head())


372

,LAT,LON
0,22.0,25.000
5,22.0,35.000
10,22.0,25.625
15,22.0,35.625
20,22.0,26.250


118

,LAT,LON
0,22.5,24.5
5,22.5,34.5
10,22.5,25.5
15,22.5,35.5
20,22.5,26.5


In [6]:
display(df1)
display(df2)

,Unnamed: 0,LAT,LON,YEAR,DOY,EVLAND,GWETPROF,GWETROOT,GWETTOP,PRECSNO,...,TS_MIN,WD2M,WD50M,WS2M,WS2M_MAX,WS2M_MIN,WS50M,WS50M_MAX,WS50M_MIN,Z0M
0,0,22.0,25.000,2019,1,0.00,0.25,0.25,0.06,0.0,...,2.63,337.8,341.6,2.36,4.76,1.08,4.81,7.19,2.92,0.01
1,0,22.0,25.000,2020,1,0.00,0.25,0.25,0.07,0.0,...,3.64,83.8,74.2,2.79,5.12,0.18,5.40,9.29,0.27,0.01
2,0,22.0,25.000,2021,1,0.00,0.25,0.25,0.05,0.0,...,4.74,90.6,95.3,2.06,3.65,1.12,5.02,7.32,2.18,0.01
3,0,22.0,25.000,2022,1,0.00,0.25,0.25,0.05,0.0,...,2.96,324.1,325.5,3.98,6.54,1.77,7.57,9.34,4.93,0.01
4,0,22.0,25.000,2023,1,0.00,0.25,0.25,0.08,0.0,...,3.34,27.1,29.0,3.07,4.45,1.81,6.74,8.48,4.73,0.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
679267,122971,32.0,31.875,2020,366,0.29,0.17,0.16,0.87,0.0,...,20.42,14.1,14.7,4.27,5.11,3.22,5.06,6.21,3.68,0.00
679268,122972,32.0,32.500,2020,366,-999.00,-999.00,-999.00,1.00,0.0,...,20.49,23.7,24.1,4.19,4.94,3.28,4.97,5.99,3.77,0.00
679269,122973,32.0,33.125,2020,366,-999.00,-999.00,-999.00,1.00,0.0,...,20.30,45.1,45.0,3.39,5.54,1.63,3.98,6.93,1.75,0.00
679270,122974,32.0,33.750,2020,366,-999.00,-999.00,-999.00,1.00,0.0,...,20.17,86.2,85.7,2.51,4.40,0.17,2.89,5.31,0.20,0.00


,Unnamed: 0,LAT,LON,YEAR,DOY,AIRMASS,ALLSKY_SFC_SW_DWN,ALLSKY_SFC_UVA,ALLSKY_SFC_UVB,ALLSKY_SFC_UV_INDEX,CLOUD_AMT,CLOUD_AMT_DAY,CLOUD_AMT_NIGHT,CLRSKY_DAYS,MIDDAY_INSOL,PSH,PW
0,0,22.5,24.5,2019,1,3.87,15.71,0.85,0.02,1.14,37.06,47.33,28.38,0.0,63.47,0.38,0.70
1,0,22.5,24.5,2020,1,3.87,16.53,0.90,0.02,1.16,22.80,9.57,34.00,0.0,66.77,0.40,1.05
2,0,22.5,24.5,2021,1,3.86,17.93,0.96,0.02,1.23,14.62,15.97,13.48,0.0,68.26,0.43,0.66
3,0,22.5,24.5,2022,1,3.86,16.45,0.90,0.02,1.14,10.43,13.11,8.17,0.0,61.22,0.39,1.08
4,0,22.5,24.5,2023,1,3.87,15.34,0.77,0.02,1.11,4.05,5.34,2.95,1.0,66.04,0.37,0.69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222767,40255,31.5,30.5,2020,366,3.81,12.20,0.64,0.01,0.54,27.96,22.33,31.97,0.0,48.22,0.29,1.42
222768,40256,31.5,31.5,2020,366,3.82,11.29,0.60,0.01,0.51,39.59,36.89,41.52,0.0,44.76,0.27,1.35
222769,40257,31.5,32.5,2020,366,3.85,12.00,0.61,0.01,0.51,17.16,11.07,21.51,0.0,48.89,0.29,1.29
222770,40258,31.5,33.5,2020,366,3.92,11.91,0.61,0.01,0.50,17.98,11.71,22.46,0.0,48.74,0.29,1.29


In [ ]:
egypt_boundary = gpd.read_file("https://raw.githubusercontent.com/datasets/geo-boundaries-world-110m/master/countries.geojson")

# Filter Egypt's boundary
egypt_polygon = egypt_boundary[egypt_boundary['admin'] == 'Egypt'].geometry.iloc[0]
coords = list(egypt_polygon.exterior.coords)
updated_coords = coords 
updated_egypt_polygon = Polygon(updated_coords)


In [ ]:
egypt_map = folium.Map(location=[26.5, 30], zoom_start=6)
points_within_polygon = []
for _, row in df1_unigue.iterrows():
    lat, lon = row['LAT'], row['LON']
    
    
    point = gpd.points_from_xy([lon], [lat])[0]
    
    if updated_egypt_polygon.contains(point): 
        points_within_polygon.append((lat, lon)) 
        # Add rectangle
        folium.CircleMarker(
    location=[lat, lon],
    radius=4,  
    color='blue',
    fill=True,
    fill_color='blue',
    fill_opacity=0.8,
    tooltip=f"Lat: {lat}, Lon: {lon}",
    popup=f"Coordinates: Lat {lat}, Lon {lon}"
).add_to(egypt_map)


specific_points = [(33.75, 27), (33.75, 27.5),(33.75,31.0),(34.375,26.0),(26.875,31.5),(28.75,31.0),(35.0,25.0),(35.625,23.5),(34.375,28.0),(34.375,28.5),(25.0,31.5),(32.5,29.5),(31.875,31.5)]

for lon, lat in specific_points:
    folium.CircleMarker(
    location=[lat, lon],
    radius=4,
    color='red',
    fill=True,
    fill_color='blue',
    fill_opacity=0.8,
    tooltip=f"Lat: {lat}, Lon: {lon}",
    popup=f"Coordinates: Lat {lat}, Lon {lon}"
).add_to(egypt_map)

egypt_map


In [ ]:
total_points_in_Egypt=points_within_polygon+specific_points

df3 = pd.DataFrame(total_points_in_Egypt, columns=['Latitude', 'Longitude'])

display(df3)

,Latitude,Longitude
0,22.500,35.000
1,22.500,35.625
2,22.500,36.250
3,23.000,35.000
4,23.000,35.625
...,...,...
284,34.375,28.000
285,34.375,28.500
286,25.000,31.500
287,32.500,29.500


In [ ]:
egypt_map_small = folium.Map(location=[26.5, 30], zoom_start=6)
points_within_polygon_small = []
for _, row in df2_unigue.iterrows():
    lat, lon = row['LAT'], row['LON']
    
    point_small = gpd.points_from_xy([lon], [lat])[0]
    
    if updated_egypt_polygon.contains(point_small): 
        points_within_polygon_small.append((lat, lon)) 

        folium.CircleMarker(
    location=[lat, lon],
    radius=4, 
    color='blue',
    fill=True,
    fill_color='blue',
    fill_opacity=0.8,
    tooltip=f"Lat: {lat}, Lon: {lon}",
    popup=f"Coordinates: Lat {lat}, Lon {lon}"
).add_to(egypt_map_small)

specific_points_small = [(27.5,31.5),(32.5,29.5),(31.5,31.5),(32.5,31.5),(33.5,27.5),(35.5,24.5)]

for lon, lat in specific_points_small:
    folium.CircleMarker(
    location=[lat, lon],
    radius=4, 
    color='red',
    fill=True,
    fill_color='blue',
    fill_opacity=0.8,
    tooltip=f"Lat: {lat}, Lon: {lon}",
    popup=f"Coordinates: Lat {lat}, Lon {lon}"
).add_to(egypt_map_small)

#egypt_map_small.save("egypt_grid_map_small.html")

egypt_map_small


In [ ]:
total_points_in_Egypt_small=points_within_polygon_small+specific_points_small

df5 = pd.DataFrame(total_points_in_Egypt_small, columns=['Latitude', 'Longitude'])

print(df5)

    Latitude  Longitude
0       22.5       34.5
1       22.5       25.5
2       22.5       35.5
3       22.5       26.5
4       22.5       27.5
..       ...        ...
91      32.5       29.5
92      31.5       31.5
93      32.5       31.5
94      33.5       27.5
95      35.5       24.5

[96 rows x 2 columns]


In [ ]:
df_filtered_small = df2[df2.apply(lambda row: (row['LAT'], row['LON']) in zip(df5['Latitude'], df5['Longitude']), axis=1)]

print("Filtered DataFrame (rows matching Latitude and Longitude pairs in df3):")
print(df_filtered_small)

Filtered DataFrame (rows matching Latitude and Longitude pairs in df3):
        Unnamed: 0   LAT   LON  YEAR  DOY  AIRMASS  ALLSKY_SFC_SW_DWN  \
5                0  22.5  34.5  2019    1     4.11              13.55   
6                0  22.5  34.5  2020    1     4.11              14.45   
7                0  22.5  34.5  2021    1     4.10              17.54   
8                0  22.5  34.5  2022    1     4.11              17.67   
9                0  22.5  34.5  2023    1     4.11              13.29   
...            ...   ...   ...   ...  ...      ...                ...   
222760       40248  30.5  34.5  2020  366     3.81              13.17   
222762       40250  31.5  25.5  2020  366     6.22              13.14   
222763       40251  31.5  26.5  2020  366     4.09              11.37   
222767       40255  31.5  30.5  2020  366     3.81              12.20   
222768       40256  31.5  31.5  2020  366     3.82              11.29   

        ALLSKY_SFC_UVA  ALLSKY_SFC_UVB  ALLSKY_SFC_

In [ ]:
df_sorted_small = df_filtered_small.sort_values(by=['YEAR', 'DOY',"LAT","LON"], ascending=[True, True,True,True])

print(df_sorted_small)

df_sorted_small.to_csv('data/sorted_data_small.csv', index=False)


        Unnamed: 0   LAT   LON  YEAR  DOY  AIRMASS  ALLSKY_SFC_SW_DWN  \
10               1  22.5  25.5  2019    1     3.93              14.94   
20               2  22.5  26.5  2019    1     4.02              13.70   
30               3  22.5  27.5  2019    1     4.14              12.29   
40               4  22.5  28.5  2019    1     4.29              11.38   
50               5  22.5  29.5  2019    1     4.46              11.21   
...            ...   ...   ...   ...  ...      ...                ...   
222606       40138  30.5  34.5  2023  365     3.83              12.66   
222616       40140  31.5  25.5  2023  365     6.24              11.73   
222621       40141  31.5  26.5  2023  365     4.09              12.07   
222641       40145  31.5  30.5  2023  365     3.83              11.22   
222646       40146  31.5  31.5  2023  365     3.83              10.68   

        ALLSKY_SFC_UVA  ALLSKY_SFC_UVB  ALLSKY_SFC_UV_INDEX  CLOUD_AMT  \
10                0.81            0.02           

In [ ]:

df_filtered_large = df1[df1.apply(lambda row: (row['LAT'], row['LON']) in zip(df3['Latitude'], df3['Longitude']), axis=1)]
print("Filtered DataFrame (rows matching Latitude and Longitude pairs in df3):")
print(df_filtered_large)

Filtered DataFrame (rows matching Latitude and Longitude pairs in df3):
        Unnamed: 0   LAT     LON  YEAR  DOY  EVLAND  GWETPROF  GWETROOT  \
45               4  22.5  35.000  2019    1    0.03      0.33      0.33   
46               4  22.5  35.000  2020    1    0.15      0.33      0.33   
47               4  22.5  35.000  2021    1    0.13      0.33      0.33   
48               4  22.5  35.000  2022    1    0.28      0.34      0.33   
49               4  22.5  35.000  2023    1    0.43      0.34      0.34   
...            ...   ...     ...   ...  ...     ...       ...       ...   
679237      122941  31.0  33.125  2020  366    0.13      0.18      0.17   
679241      122945  31.5  25.625  2020  366    0.94      0.57      0.59   
679242      122946  31.5  26.250  2020  366    1.12      0.59      0.62   
679249      122953  31.5  30.625  2020  366    0.57      0.50      0.49   
679250      122954  31.5  31.250  2020  366    0.36      0.61      0.60   

        GWETTOP  PRECSNO  .

In [ ]:

df_sorted = df_filtered_large.sort_values(by=['YEAR', 'DOY',"LAT","LON"], ascending=[True, True,True,True])

print(df_sorted)

df_sorted.to_csv('data/sorted_data_large.csv', index=False)


        Unnamed: 0   LAT     LON  YEAR  DOY  EVLAND  GWETPROF  GWETROOT  \
170             17  22.5  25.625  2019    1    0.00      0.37      0.36   
180             18  22.5  26.250  2019    1    0.00      0.27      0.27   
190             19  22.5  26.875  2019    1    0.00      0.20      0.20   
200             20  22.5  27.500  2019    1    0.00      0.15      0.15   
210             21  22.5  28.125  2019    1    0.00      0.20      0.20   
...            ...   ...     ...   ...  ...     ...       ...       ...   
678765      122605  31.0  33.125  2023  365    0.13      0.14      0.14   
678785      122609  31.5  25.625  2023  365    0.14      0.35      0.35   
678790      122610  31.5  26.250  2023  365    0.09      0.34      0.33   
678825      122617  31.5  30.625  2023  365    0.92      0.51      0.51   
678830      122618  31.5  31.250  2023  365    0.98      0.64      0.64   

        GWETTOP  PRECSNO  ...  TS_MIN   WD2M  WD50M  WS2M  WS2M_MAX  WS2M_MIN  \
170        0.09   

In [ ]:
shapefile_path = "City_ADM\EGY_adm2.shp"
cities = gpd.read_file(shapefile_path).to_crs("EPSG:3857") 
def find_nearest_city(point, cities):
    """Find the nearest city to a given point (in projected CRS)."""
    distances = cities.distance(point)
    nearest_idx = distances.idxmin()
    return cities.loc[nearest_idx, city_column] if nearest_idx in cities.index else None

def find_nearest_governorate(point, cities_gdf):
    """Find the governorate of the nearest city for a given point."""
    distances = cities_gdf.geometry.distance(point)
    nearest_idx = distances.idxmin()
    if pd.notna(nearest_idx):
        return cities_gdf.loc[nearest_idx, "NAME_1"] 
    return None 


FILES=["large.csv","small.csv"]
for i in FILES:
    df = pd.read_csv("data/sorted_data_{}".format(i))

    df["geometry"] = df.apply(lambda row: Point(row["LON"], row["LAT"]), axis=1)
    geo_df = gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326")

    geo_df = geo_df.to_crs("EPSG:3857")

    result = gpd.sjoin(geo_df, cities, how="left", predicate="within")

    city_column = "NAME_2" 
    result = result.rename(columns={city_column: "City"})
    GOV_column = "NAME_1" 
    result = result.rename(columns={GOV_column: "GOV"})
    
    
    missing_city_mask = result["City"].isna()
    if missing_city_mask.any():
        print(f"Fixing {missing_city_mask.sum()} missing locations...")
        result.loc[missing_city_mask, "City"] = result.loc[missing_city_mask, "geometry"].apply(lambda x: find_nearest_city(x, cities))
        
    missing_gov_mask = result["GOV"].isna()
    if missing_gov_mask.any():
        print(f"Fixing {missing_gov_mask.sum()} missing governorates...")
        result.loc[missing_gov_mask, "GOV"] = result.loc[missing_gov_mask, "geometry"].apply(
            lambda x: find_nearest_governorate(x, cities)
        )

    result["Date"] = pd.to_datetime(result["YEAR"].astype(str) + result["DOY"].astype(str), format='%Y%j')
    result["Week"] = result["Date"].dt.isocalendar().week  
    result["Month"] = result["Date"].dt.month
    print("Missing governorates after fixing:", result["GOV"].isna().sum())    

Fixing 16434 missing locations...
Fixing 16434 missing governorates...
Missing governorates after fixing: 0
Fixing 9130 missing locations...
Fixing 9130 missing governorates...
Missing governorates after fixing: 0


In [ ]:
FILES=["large.csv","small.csv"]
for i in FILES:
    df = pd.read_csv("data/fullWithLocations_{}".format(i))
    display(df.columns)
    df = df.drop(columns=["geometry", "index_right", "ID_0", "ISO", "NAME_0", "ID_1", "ID_2", "TYPE_2", "ENGTYPE_2", "NL_NAME_2", "VARNAME_2"])
    df.to_csv("data/fullWithLocations_FINAL_{}".format(i), index=False)

Index(['Unnamed: 0', 'LAT', 'LON', 'YEAR', 'DOY', 'EVLAND', 'GWETPROF',
       'GWETROOT', 'GWETTOP', 'PRECSNO', 'PRECTOTCORR', 'QV2M', 'RH2M', 'RHOA',
       'T10M', 'T10M_MAX', 'T10M_MIN', 'T2M', 'T2M_MAX', 'T2M_MIN', 'TO3',
       'TS', 'TSOIL1', 'TSOIL2', 'TSOIL3', 'TSOIL4', 'TSOIL5', 'TSOIL6',
       'TS_MAX', 'TS_MIN', 'WD2M', 'WD50M', 'WS2M', 'WS2M_MAX', 'WS2M_MIN',
       'WS50M', 'WS50M_MAX', 'WS50M_MIN', 'Z0M', 'geometry', 'index_right',
       'ID_0', 'ISO', 'NAME_0', 'ID_1', 'GOV', 'ID_2', 'City', 'TYPE_2',
       'ENGTYPE_2', 'NL_NAME_2', 'VARNAME_2', 'Date', 'Week', 'Month'],
      dtype='object')

Index(['Unnamed: 0', 'LAT', 'LON', 'YEAR', 'DOY', 'AIRMASS',
       'ALLSKY_SFC_SW_DWN', 'ALLSKY_SFC_UVA', 'ALLSKY_SFC_UVB',
       'ALLSKY_SFC_UV_INDEX', 'CLOUD_AMT', 'CLOUD_AMT_DAY', 'CLOUD_AMT_NIGHT',
       'CLRSKY_DAYS', 'MIDDAY_INSOL', 'PSH', 'PW', 'geometry', 'index_right',
       'ID_0', 'ISO', 'NAME_0', 'ID_1', 'GOV', 'ID_2', 'City', 'TYPE_2',
       'ENGTYPE_2', 'NL_NAME_2', 'VARNAME_2', 'Date', 'Week', 'Month'],
      dtype='object')